In [1]:
import warnings
warnings.filterwarnings("ignore")
import sys, os
sys.path.append(os.path.abspath("../.."))
from configs import GOOGLE_APPLICATION_CREDENTIALS,GCS_BUCKET_NAME,GCS_PROJECT_ID
from google.cloud import bigquery
from src.utils.io_utils import upload_to_bigquery
from clean_utils import *

In [2]:
client = bigquery.Client.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)
table_id = f"{GCS_PROJECT_ID}.{GCS_BUCKET_NAME}.data_cleaned"

In [3]:
query = """SELECT *
FROM `khangtestdbt.xecupredict.chotot_com` """
data_chotot = client.query(query).to_dataframe()
data_chotot.head(1)

,name,price,Số_Km_đã_đi,Còn_hạn_đăng_kiểm,Xuất_xứ,Tình_trạng,Chính_sách_bảo_hành,Hãng,Dòng_xe,Năm_sản_xuất,Hộp_số,Nhiên_liệu,Số_chỗ,Trọng_lượng,Trọng_tải,Số_đời_chủ,Có_phụ_kiện_đi_kèm,Kiểu_dáng
0,Hyundai Accent 2022 - 65000,410.000.000 đ,50000,None,Việt Nam,Đã sử dụng,Bảo hành hãng,Hyundai,Accent,2022,Tự động,Xăng,5,> 1 tấn,> 2 tấn,1 chủ,None,None


In [4]:
df_chotot = data_chotot.copy()

In [5]:
df_chotot.columns

Index(['name', 'price', 'Số_Km_đã_đi', 'Còn_hạn_đăng_kiểm', 'Xuất_xứ',
       'Tình_trạng', 'Chính_sách_bảo_hành', 'Hãng', 'Dòng_xe', 'Năm_sản_xuất',
       'Hộp_số', 'Nhiên_liệu', 'Số_chỗ', 'Trọng_lượng', 'Trọng_tải',
       'Số_đời_chủ', 'Có_phụ_kiện_đi_kèm', 'Kiểu_dáng'],
      dtype='object')

In [6]:
df_chotot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8937 entries, 0 to 8936
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name                 8937 non-null   object
 1   price                8937 non-null   object
 2   Số_Km_đã_đi          8102 non-null   object
 3   Còn_hạn_đăng_kiểm    4041 non-null   object
 4   Xuất_xứ              8937 non-null   object
 5   Tình_trạng           8937 non-null   object
 6   Chính_sách_bảo_hành  8937 non-null   object
 7   Hãng                 8937 non-null   object
 8   Dòng_xe              8937 non-null   object
 9   Năm_sản_xuất         8937 non-null   object
 10  Hộp_số               8937 non-null   object
 11  Nhiên_liệu           8937 non-null   object
 12  Số_chỗ               7860 non-null   object
 13  Trọng_lượng          8102 non-null   object
 14  Trọng_tải            8102 non-null   object
 15  Số_đời_chủ           3983 non-null   object
 16  Có_phụ

In [7]:
df_chotot = df_chotot[["Số_Km_đã_đi", "Tình_trạng", "Xuất_xứ", "Kiểu_dáng", "Nhiên_liệu", "name", "price", "Hãng", "Năm_sản_xuất"]].copy()

In [8]:
df_chotot.columns = ["km", "Tình_trạng", "origin", "body", "fuel", "name", "price", "brand", "age"]

In [9]:
df_chotot.describe(include='all').T

,count,unique,top,freq
km,8102,1243,100000,261
Tình_trạng,8937,2,Đã sử dụng,8102
origin,8937,11,Việt Nam,3777
body,7109,9,SUV / Cross over,2693
fuel,8937,4,Xăng,6780
name,8937,8799,VinFast Limo Green 2025,6
price,8937,1207,495.000.000 đ,83
brand,8937,63,Toyota,1804
age,8937,42,2025,1051


In [10]:
df_chotot.isna().sum()

km             835
Tình_trạng       0
origin           0
body          1828
fuel             0
name             0
price            0
brand            0
age              0
dtype: int64

1.name

In [11]:
df_chotot["name"] = df_chotot["name"].apply(clean_name)
df_chotot.head(1)

,km,Tình_trạng,origin,body,fuel,name,price,brand,age
0,50000,Đã sử dụng,Việt Nam,None,Xăng,Hyundai Accent 2022 65000,410.000.000 đ,Hyundai,2022


2.price

In [12]:
df_chotot["price"].sample(5)

2183    1.069.000.000 đ
2484      719.000.000 đ
4248      248.000.000 đ
6804      615.000.000 đ
8686    1.195.000.000 đ
Name: price, dtype: object

In [13]:
df_chotot["price"] = (
    df_chotot["price"]
    .str.replace("đ", "", regex=False)
    .str.replace(".", "", regex=False)
    .str.strip()
    .astype(float)
    .astype("Int64")
)
df_chotot.price.sample(5)

8316    510000000
4962    380000000
3030    790000000
5952    415000000
3817    755000000
Name: price, dtype: Int64

In [14]:
df_chotot["price"].isna().sum()

np.int64(0)

In [15]:
df_chotot = df_chotot[df_chotot["price"] > 50000000]

3.brand

In [16]:
df_chotot["brand"].sample(5)

7909    Mercedes Benz
1721           Toyota
1256              Kia
6857          VinFast
6442           Toyota
Name: brand, dtype: object

In [17]:
df_chotot["brand"].isna().sum()

np.int64(0)

In [18]:
df_chotot["brand"].unique()

array(['Hyundai', 'Chevrolet', 'VinFast', 'Toyota', 'Ford', 'Kia',
       'Suzuki', 'Mitsubishi', 'Honda', 'Nissan', 'Mazda',
       'Mercedes Benz', 'Subaru', 'MG', 'Peugeot', 'Daewoo', 'Isuzu',
       'Wuling', 'BYD', 'Skoda', 'Volkswagen', 'Citroen', 'Audi', 'MINI',
       'BMW', 'Lexus', 'GAC', 'Ferrari', 'Volvo', 'Luxgen', 'Porsche',
       'Hãng khác', 'Rolls Royce', 'LandRover', 'Lynk&Co', 'Acura',
       'Lada', 'Fiat', 'Jeep', 'Daihatsu', 'Geely', 'Alfa Romeo',
       'Dongfeng', 'Asia', 'Lamborghini', 'Chery', 'Ssangyong', 'Samsung',
       'SYM', 'Jaguar', 'Chrysler', 'Bentley', 'Smart', 'Gaz', 'Maserati',
       'Cadillac', 'Zotye', 'Changan', 'Mekong', 'Reult', 'Baic',
       'Genesis'], dtype=object)

In [19]:
df_chotot["brand"] = df_chotot["brand"].str.lower()

4.year

In [20]:
df_chotot["age"].isna().sum()

np.int64(0)

In [22]:
df_chotot["age"] = pd.to_numeric(df_chotot["age"], errors="coerce")

In [23]:
df_chotot = df_chotot.dropna(subset=["age"])

In [24]:
df_chotot["age"] = 2025 - df_chotot["age"].astype(int)

In [25]:
df_chotot.sample(5)

,km,Tình_trạng,origin,body,fuel,name,price,brand,age
3571,160000,Đã sử dụng,Đài Loan,Hatchback,Xăng,Mazda 3 2010 1 6 MT 160000 km,220000000,mazda,15
6143,85000,Đã sử dụng,Việt Nam,None,Xăng,Mazda CX 5 2018 Đỏ 85000 km,555000000,mazda,7
667,57000,Đã sử dụng,Việt Nam,Sedan,Xăng,Honda City Top 2019 Đen 57000 km xe gia đình,395000000,honda,6
7308,32000,Đã sử dụng,Việt Nam,SUV / Cross over,Xăng,Mazda CX5 2020 2 0 Premium,695000000,mazda,5
1227,77000,Đã sử dụng,Việt Nam,Pick-up (bán tải),Dầu,Ford Ranger XLS 2021 2 2L 4x2 AT 77000km,558000000,ford,4


In [26]:
df_chotot["age"].isna().sum()

np.int64(0)

5.status

In [27]:
df_chotot["Tình_trạng"].unique()

array(['Đã sử dụng', 'Mới'], dtype=object)

In [28]:
df_chotot = df_chotot[df_chotot["Tình_trạng"] == "Đã sử dụng"]
df_chotot.drop(columns=["Tình_trạng"], inplace=True)

6.km

In [29]:
df_chotot["km"].head(5)

0    50000
1        1
3    10000
4    32000
5        0
Name: km, dtype: object

In [30]:
df_chotot["km"].isna().sum()

np.int64(0)

In [31]:
df_chotot["km"].describe()

count       7930
unique      1216
top       100000
freq         248
Name: km, dtype: object

In [32]:
df_chotot["km"] = df_chotot["km"].astype(int)

In [33]:
df_chotot = df_chotot[df_chotot["km"] > 100]

In [34]:
df_chotot.head()

,km,origin,body,fuel,name,price,brand,age
0,50000,Việt Nam,None,Xăng,Hyundai Accent 2022 65000,410000000,hyundai,3
3,10000,Việt Nam,None,Xăng,Chevrolet Cruze 2011 1 6 sàn Bạc,158000000,chevrolet,14
4,32000,Việt Nam,None,Xăng,VinFast Fadil 2022 1 4 Đỏ 32000 km,315000000,vinfast,3
8,102132,Đang cập nhật,None,Xăng,Chevrolet Spark 2015 Đỏ Số tự động,172000000,chevrolet,10
9,100000,Đang cập nhật,None,Xăng,Chevrolet Spark Lite Bạc,55000000,chevrolet,14


7.origin

In [35]:
df_chotot["origin"].unique()

array(['Việt Nam', 'Đang cập nhật', 'Hàn Quốc', 'Nước khác', 'Nhật Bản',
       'Thái Lan', 'Trung Quốc', 'Mỹ', 'Ấn Độ', 'Đức', 'Đài Loan'],
      dtype=object)

In [36]:
df_chotot["origin"] = df_chotot["origin"].replace({
    "Việt Nam": "trong nước"
}).apply(lambda x: "nhập khẩu" if x not in ["trong nước"] else x)

In [37]:
df_chotot["origin"].value_counts()

origin
nhập khẩu     4529
trong nước    3131
Name: count, dtype: int64

8.body

In [38]:
df_chotot["body"].isna().sum()

np.int64(1546)

In [39]:
df_chotot["body"].unique()

array([None, 'Hatchback', 'Minivan (MPV)', 'Pick-up (bán tải)',
       'SUV / Cross over', 'Sedan', 'Van', 'Kiểu dáng khác',
       'Coupe (2 cửa)', 'Mui trần'], dtype=object)

In [40]:
df_chotot["body"] = df_chotot["body"].str.lower().str.strip()

9.fuel

In [41]:
df_chotot["fuel"].isna().sum()

np.int64(0)

In [42]:
df_chotot["fuel"].unique()

array(['Xăng', 'Dầu', 'Động cơ Hybrid', 'Điện'], dtype=object)

In [43]:
df_chotot["fuel"] = df_chotot["fuel"].apply(clean_fuel)

In [44]:
df_chotot["fuel"].unique()

array(['Xăng', 'Dầu', 'Hybrid', 'Điện'], dtype=object)

10.updata

In [45]:
df_chotot.sample(10)

,km,origin,body,fuel,name,price,brand,age
6368,185078,trong nước,minivan (mpv),Xăng,2008 G 185078 km,168000000,toyota,17
4608,150000,nhập khẩu,None,Xăng,Ford Mondeo 2009 dk 2014 2 3 STD 6 cấp,210000000,ford,16
1604,91000,trong nước,sedan,Xăng,Bán xe Mercedes C250 Exclusive 2016 91000 km,658000000,mercedes benz,9
6759,260000,trong nước,van,Dầu,Ford Transit 2011 3 chỗ 940kg mới hạ máy zin,240000000,ford,14
3735,99000,trong nước,sedan,Xăng,Mercedes Benz C Class 2010 C200 CGI Giá 199tr,199000000,mercedes benz,15
2083,100000,trong nước,sedan,Xăng,Ford Fiesta 2014 1 5 AT Titanium 100000 km,249000000,ford,11
8605,79000,nhập khẩu,pick-up (bán tải),Dầu,Ford Ranger 2021 XLS 2 2L 4x2 MT 79000 km,530000000,ford,4
5088,83000,trong nước,sedan,Xăng,Toyota Vios 2020 G full option trắng đẹp,369000000,toyota,5
6894,8000,trong nước,sedan,Xăng,Toyota Vios 2016 1 5G 8000 km,230000000,toyota,9
6830,93000,trong nước,suv / cross over,Xăng,Chevrolet Vivant 2010 số tự động 7 chổ,159000000,chevrolet,15


In [46]:
upload_to_bigquery(df_chotot, table_id=table_id, if_exists="append" )

✅ Uploaded 7660 rows to khangtestdbt.xecupredict.data_cleaned


In [43]:
#end